In [1]:
%azure.connect "/subscriptions/b3ed2c3e-e784-4a4e-9474-a3d279d2f9e4/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/LaboratorioQuantum-Ximena-Toledo-RIvera" location="eastus"


Authenticated using Microsoft.Azure.Quantum.Authentication.TokenFileCredential


Connected to Azure Quantum workspace LaboratorioQuantum-Ximena-Toledo-RIvera in location eastus.


Target ID,Current Availability,Average Queue Time (Seconds)
microsoft.estimator,Available,0


# Shor Algorithm
## Phase 1 

In [3]:
open Microsoft.Quantum.Canon;
    open Microsoft.Quantum.Intrinsic;
    open Microsoft.Quantum.Measurement;
    open Microsoft.Quantum.Math;
    open Microsoft.Quantum.Convert;
    open Microsoft.Quantum.Arithmetic;
    open Microsoft.Quantum.Oracles;
    open Microsoft.Quantum.Random;
    open Microsoft.Quantum.Diagnostics;
    open Microsoft.Quantum.Characterization;

operation ramdomNumberGenerator() : Result {
        use q1 = Qubit();
        H(q1);
        return M(q1);
    }

ramdomNumberGenerator

In [8]:
%simulate ramdomNumberGenerator

One

In [9]:
operation randomNumberRange(min: Int, max: Int) : Int {
        mutable output = 0;
        repeat { // Loop to generate random numbers until it generates one that's equal or less than max or equal or more than min
            mutable bits = [];
            for indexBit in 1..BitSizeI(max) { // Returns the number of bits required
                set bits = bits + [ramdomNumberGenerator()];
            }
            for indexBit in 1..BitSizeI(min) {
                set bits = bits + [ramdomNumberGenerator()];
            }
            set output = ResultArrayAsInt(bits);
        } until (output >= min and output <= max );
        Message($"Random nuIntmber between {min} and {max} ->  ");
        return output;
    }

randomNumberRange

In [10]:
%simulate randomNumberRange min=100 max=200

Random nuIntmber between 100 and 200 ->  


147

In [11]:
%simulate randomNumberRange min=100 max=200

Random nuIntmber between 100 and 200 ->  


137

In [12]:
%simulate randomNumberRange min=100 max=200

Random nuIntmber between 100 and 200 ->  


145

In [13]:
%simulate randomNumberRange min=100 max=200

Random nuIntmber between 100 and 200 ->  


103

In [14]:
%simulate randomNumberRange min=100 max=200

Random nuIntmber between 100 and 200 ->  


142

In [15]:
%simulate randomNumberRange min=100 max=200

Random nuIntmber between 100 and 200 ->  


115

## Euclidean Algorithm 

In [16]:
function euclideanAlgoGCD(a: Int, N: Int) : Int {
        Message($"GCD of {a} and {N} ->  ");
        return GreatestCommonDivisorI((a),(N));
}

euclideanAlgoGCD

In [17]:
%simulate euclideanAlgoGCD a=60 N=48

GCD of 60 and 48 ->  


12

In [18]:
operation euclideanAlgoGCDIm(a: Int, N: Int) : Int {
        if((N) == 0) {
            return a;
        }  Message($"GCD of {a} -> "); 
        return euclideanAlgoGCDIm((N), (a) % (N));
    }

euclideanAlgoGCDIm

In [19]:
%simulate euclideanAlgoGCDIm a=60 N=48

GCD of 60 -> 
GCD of 48 -> 


12

In [20]:
operation euclideanAlgoGCDImp2(a: Int, N: Int) : Int {
        if((a) == 0) {
            return (N);
        } if ((N) == 0) {
            return (a);
        } if ((a) == (N)) {
            return (a);
        } if ((a) > (N)) {
            Message($"GCD of {a} and {N} -> "); 
            return euclideanAlgoGCDImp2((a)-(N), (N));
        }
        return euclideanAlgoGCDImp2((a), (N)-(a));
    }

euclideanAlgoGCDImp2

In [21]:
%simulate euclideanAlgoGCDImp2 a=60 N=48

GCD of 60 and 48 -> 


12

In [22]:
operation possibleFactorsR (module: Int, randomGenerator: Int, r:Int) : (Bool, (Int, Int)) {
        if r % 2 == 0 {
            let halfExponentiation = ExpModI(randomGenerator, r/2, module);
            if halfExponentiation != module - 1 {
                let factor = MaxI (
                    GreatestCommonDivisorI(halfExponentiation - 1, module),
                    GreatestCommonDivisorI(halfExponentiation + 1, module)
                );
                return (true, (factor, module / factor));
            } else {
                return (false, (1,1));
            }
        } else {
        return(false, (1,1));
        } 
    }

possibleFactorsR

In [25]:
%simulate possibleFactorsR module=2 randomGenerator=20 r=4
// testeng 1 

(True, (1, 2))

In [26]:
operation applyFindingOracle (randomGenerator : Int, module: Int, power : Int, target: Qubit[] ) : Unit is Adj + Ctl{
        Fact(IsCoprimeI(randomGenerator,module),"The guess number is `randomGenerator`, and the mudule is `module`.");

        // ! https://learn.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.arithmetic.multiplybymodularinteger

        // Performing modular multiplication by an integer constant on a qubit register.
        MultiplyByModularInteger(ExpModI(randomGenerator, power, module),module, LittleEndian(target));
    }

applyFindingOracle

In [52]:
operation periodFrequency (module : Int, frequencyEstimation : Int, bitsPrecision: Int, currentDivisor: Int) : Int {
        // ! https://learn.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.math.continuedfractionconvergenti
        // Finds the continued fraction convergent closest to fraction with the denominator less or equal to denominatorBound
        let (s, r) = (ContinuedFractionConvergentI(Fraction(frequencyEstimation, 2 ^ bitsPrecision), module))!;
        let (sAbsolute, rAbsolute) = (AbsI(s), AbsI(r));
        //  Computes the greatest common divisor of two integers.
        return (rAbsolute * currentDivisor) / GreatestCommonDivisorI(currentDivisor, rAbsolute);
    } 

periodFrequency

In [56]:
operation EstimateFrequencyValue (randomGenerator : Int, module: Int, bits : Int) : Int {   
        mutable frequencyEstimation = 0;
        // ! https://learn.microsoft.com/fr-fr/qsharp/api/qsharp/microsoft.quantum.characterization.robustphaseestimation?view=qsharp-preview
        let bitsPrecision = 2 * bits + 1;

        // ! https://learn.microsoft.com/en-us/azure/quantum/concepts-advanced-matrices
        use eigenStateRegister = Qubit[bits];
        
        // ! https://learn.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.arithmetic.littleendian
        let eigenStateRegisterLittleEndian = LittleEndian(eigenStateRegister);

        // Applying a bitwise-XOR operation between a classical integer and an integer represented by a register of qubits.
        ApplyXorInPlace(1, eigenStateRegisterLittleEndian);
        let oracle = applyFindingOracle(randomGenerator, module, _, _);

        // * QFT to estimate the frequency
        use a = Qubit();
        for index in bitsPrecision -1..-1..0 {
            within {
                H(a);
            } apply {
                Controlled oracle([a], (1 <<< index, eigenStateRegisterLittleEndian!));
                // Applying a rotation about the |1> state by an angle specified as a dyadic fraction.
                // ! https://learn.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.r1frac
                R1Frac(frequencyEstimation, bitsPrecision - 1 - index, a);
            } if MResetZ(a) == One {
                set frequencyEstimation += 1 <<< (bitsPrecision - 1 - index);
            }
        }
        // Measuring the qubits and ensure they are in the |0⟩ state such that they can be safely released.
        ResetAll(eigenStateRegister);
        return frequencyEstimation;
    }

EstimateFrequencyValue

In [57]:
operation phaseEstimation (randomGenerator: Int, module: Int ) : Int {
        // ! https://learn.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.fact
        Fact(IsCoprimeI(randomGenerator,module),"The guess number is `randomGenerator`, and the mudule is `module`.");

        // Storing the r found. 
        mutable result = 1;

        // Number of bits in the mudule 
        let bits = BitSizeI(module);

        let bitsPrecision = 2 * bits + 1;

        mutable frequencyEstimation = 0;

        set frequencyEstimation = EstimateFrequencyValue (randomGenerator, module,bits);
        
        if  frequencyEstimation != 0 {
            set result = periodFrequency (module, frequencyEstimation, bitsPrecision, result);
        } else {
            Message("Estimated frecuency was 0.");
        }
        return result;
    }

phaseEstimation

In [71]:
operation shorImplementatonTest (n: Int): (Int, Int) {
        // Trivial case: Even number
        if n % 2 == 0 {
            // Checking if there is a even number.
            Message("Even number.");
            return (n/2, 2);
        }
        
        // Set up the unknown factors and set the default values.
        mutable setUpFactors = false;
        mutable defaultFactors = (1,1); 

        // ! https://learn.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.random.drawrandomint
        repeat {
            let randomGenerator =  DrawRandomInt(2, n - 1); // ? IBM(2, N-1),  MICROSOT(1 < a < N-1)
            // ! https://learn.microsoft.com/es-mx/qsharp/api/qsharp/microsoft.quantum.math.iscoprimei
            if IsCoprimeI (randomGenerator,n) {
                Message($"Guess number: {randomGenerator}");
                let r = phaseEstimation(randomGenerator, n);
                set (setUpFactors,defaultFactors) = possibleFactorsR(n, randomGenerator, r);
            }  else {
                let GCD = GreatestCommonDivisorI(n,randomGenerator);
                Message($"Divisor guessed: {n} GCD: {GCD}.");
                set setUpFactors = true;
                set defaultFactors = (GCD, n/ GCD);
            }
        } until setUpFactors 
        fixup {
        Message("The estimated period did not return a valid factor.");
        }
        return defaultFactors;
    }

shorImplementatonTest

In [72]:
%simulate shorImplementatonTest n=102

Even number.


(51, 2)

In [73]:
%simulate shorImplementatonTest n=15

Divisor guessed: 15 GCD: 3.


(3, 5)

In [74]:
%simulate shorImplementatonTest n=21

Guess number: 5
The estimated period did not return a valid factor.
Guess number: 19


(3, 7)

In [75]:
%simulate shorImplementatonTest n=1004

Even number.


(502, 2)

In [76]:
%simulate shorImplementatonTest n=35

Guess number: 23


(7, 5)

In [77]:
%simulate shorImplementatonTest n=39

Guess number: 29
Estimated frecuency was 0.
The estimated period did not return a valid factor.
Guess number: 14
Estimated frecuency was 0.
The estimated period did not return a valid factor.
Divisor guessed: 39 GCD: 3.


(3, 13)